# Part6: Model deployment

data is taken from https://www.kaggle.com/c/santander-customer-transaction-prediction

In this challenge, Santander invites Kagglers to help them identify which customers will make a specific transaction in the future, irrespective of the amount of money transacted. The data provided for this competition has the same structure as the real data they have available to solve this problem.

The data is anonimyzed, each row containing 200 numerical values identified just with a number.

In this notebook we prepare file for submition to kaggle

## Load packages

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from keras import regularizers
from keras.constraints import max_norm
from keras.models import model_from_json

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import balanced_accuracy_score

Using TensorFlow backend.


## Load data

In [2]:
X_test = np.load("X_test_std.npy")
print (X_test.shape)


(200000, 200)


## Load trained model

In [3]:
# load json and create model
#name_ext = '1551635924.673968' #accuracy score:0.816, misid 0 as 1: 14%, misid 1 as 0: 23%; 
#name_ext = '1551641103.5993311' #128, accuracy score:0.856, misid 0 as 1: 7%, misid 1 as 0: 22%;
name_ext = '1551734984.7441425' #32, acc score:0.711,  misid 0 as 1: 13%, misid 1 as 0: 45%;
json_file = open("model_NN_32_ts"+name_ext+".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_NN_32 = model_from_json(loaded_model_json)
# load weights into new model
loaded_model_NN_32.load_weights("model_NN_32_ts"+name_ext+".h5")
print("Loaded model_NN_32 from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model_NN_32 from disk


## Predict results for test data

In [4]:
predictions = loaded_model_NN_32.predict(X_test)
count_0 = 0
count_1 = 0
for i in range(len(predictions)):
    if predictions[i]<0.5:
        predictions[i] = int(0)
        count_0 +=1
    else:
        predictions[i] = int(1)
        count_1 +=1


In [5]:
print("There are {}% target values with 0".format(100 * count_0/len(predictions)))
print("There are {}% target values with 1".format(100 * count_1/len(predictions)))

There are 83.1035% target values with 0
There are 16.8965% target values with 1


## Prepare file for submition
!mv sub.csv old_sub.csv


In [6]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = predictions
sub.to_csv('sub.csv', index=False)

## Conclusion, observations

* NN with 32 input layers and batch size 4*512 shows good performance already with 10 epoch (balanced_score=0.84), which becomes even better for 100 (balanced_score=0.87) and 500 epoch (balanced_score=0.88) 
